In [ ]:
"""
Logistic

Missing Value Imputation Strategies:
spending columns → 0
age -> median
categorical → 'Unknown'

New Features:
None

Deleted Features:
Name
"""

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
import missingno as msno

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

y = train["Transported"].astype(int)

X = train.drop(columns=["Transported"])
X_test = test.copy()

In [3]:
# spending columns → 0
spend_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
X[spend_cols] = X[spend_cols].fillna(0)
X_test[spend_cols] = X_test[spend_cols].fillna(0)

# Age → median
X['Age'] = X['Age'].fillna(X['Age'].median())
X_test['Age'] = X_test['Age'].fillna(X['Age'].median())

# categorical → 'Unknown'
cat_cols = ['HomePlanet','CryoSleep','Destination','Cabin','VIP']
for c in cat_cols:
    X[c] = X[c].fillna('Unknown')
    X_test[c] = X_test[c].fillna('Unknown')


In [4]:
drop_cols = ['Name', 'Cabin']
X = X.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

In [5]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

# train / test 컬럼 정렬
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)


In [6]:

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

val_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_pred))


Validation Accuracy: 0.772857964347326


/Users/heosunghak/miniforge3/envs/titanic/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
test_pred = model.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Transported": test_pred.astype(bool)
})

submission.to_csv("./submission/v1.csv", index=False)
